In [3]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split

from utils import encode_labels  # from your utils.py


In [4]:
labels_df = pd.read_csv("labels.csv")  # path to your CSV
print("Total images:", len(labels_df))

# Encode breed labels into integers
labels_df['breed_encoded'], label_encoder = encode_labels(labels_df['breed'])
num_classes = len(label_encoder.classes_)
print("Number of unique breeds:", num_classes)


Total images: 10222
Number of unique breeds: 120


In [5]:
class DogBreedDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.dataframe.iloc[idx]['id']
        breed = self.dataframe.iloc[idx]['breed_encoded']
        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(breed)


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


In [7]:
train_df, val_df = train_test_split(labels_df, test_size=0.2, stratify=labels_df['breed_encoded'], random_state=42)

train_dataset = DogBreedDataset(train_df, "train", transform=transform)
val_dataset = DogBreedDataset(val_df, "train", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))


Train samples: 8177
Val samples: 2045
